## 28/2/2024

Solution data structure
- Solution
    - "nodes": Nodes solution
    - "solution": Solution values on solution nodes
    - "elements": Elements of solution nodes
    - "elementsBoundaryNodesOnly": Elements which only connect the boundary nodes and thus are more efficient for random point sampling
    - "areaElementsBoundaryNodesOnly": Area of the elements which only connect the boundary nodes
    - *Interpolation function for the solution for the encoder: interpSE(x,y)
    - *Interpolation function for the solution for the decoder: interpSD(x,y)
- Boundary Lines
    - List of vectors of cloed boundary curves nodes indexes: [idxs1[], idxs2[], idxs3[], …] (the first curve is the outer loop, the rest are inner holes) (first element is also the last)
    - List of matrices of nodes along the boundary curves: [nodesB1[], nodesB1[], nodesB1[], ...]
    - List of vectors of distance along the line: [distances1[], distances2[], distances3[], ...]
    - List of curve lengths per line: [L1, L2, L3, ...]
    - List of vectors of normals along the line [normals1[], normals2[], normals3[], ...]
    - *List of interpolation functions for the solution along the boundary lines: [interpBS1(distance), interpBS2(distance), interpBS3(length), …]
    - *List of interpolation functions for the normals along the boundary lines: [interpBN1(length), interpBN2(length), interpBN3(distance), …]
    - *List of interpolation functions for the x,y coordinates along the boundary lines: [interpBC1(distance), interpBC2(distance), interpBC3(distance), …]
    - *Interpolation function for isInsideTheDomian: interpD(x,y)
- Boundary Corners
    - List of corner nodes indexes: [idxNode1, idxNode2, idNode3, …]
    - List of the boundary curves that include the corner: [idxCurve1, idxCurve2, idxCurve3, …]
    - List of corner indexes within the curve: [idxCornerCurve1, idxCornerCurve1, idxCornerCurve1, ...]
    - List of distances where the corners are on their respective curves [distance1, distance2, distance3, ...]


\* Not part of saved dataset but computed when loading the dataset

The boundary line direction convention
- The domain is always to the right of the line if looking in the direction of the line
    - outer boundary edge is counter-clockwise
    - inner boundary edge is clockwise
- The boundary normals always point outwards from the domain

## 29/2/2024

For the boundary interpolation Pchip interpolator used as it does not introduce oscilations around discontinuities and is a high-order (cubic) method

## 1/3/2024

The low reynodls number flow data processed to the desirable formad and uploaded to TrainingData. REQUIRES TESTING!!!

## 4/3/2024

Paper structure
General Method description
1. Key idea description
    1. Describe the boundary observation problem
    2. Describe the 3 operator approach for solving it
2. Related methods review
3. General description of the training
4. General description of Encoder
5. General description of Boundary Encoder
6. General description of Decoder
7. General description of the elliptic system
Numerical Implementation
1. Detailed description of the Encoder
2. Detailed description of the Boundary Encoder
3. Detailed description of the Decoder
4. Detailed description of the elliptic system
5. Detailed description of the Training loop
    1. Training Patches Sampling
    2. Forward pass
    3. Loss and Optimisation
6. Detailed description of solving a boundary value problem

## 13/3/2024

The interior and boundatry training sample implemented with the updated version of the solution data structure which allows to handle the fluid examples

The low reyonolds number flow dataset has small but non-zero velocity values at the boundary. For now I left it as it is but possibly should be fixed in the future

There is an inefficiency in the current implementation in interpolating the interior encoder input u_IE and b_IE. the intepSE function is also sampling b_IE internally but not returning it. It would be way more efficient to just return b_IE from interpSE. Shoud change in future

## 24/3/2024

The updated training loop was implemented without the corner training patches and applied to th low reyonodls number flow dataset. The results are promising, the SINNs model is able to qualitatively model the flow reasonably well and there are wierd artefacts around corners. 
- This is likely caused by the fact that there are no training patches around the corners and so the model will not learn how to handle them. The fix is ADDING THE CORNER TRAINING PATCHES INTO THE TRAINING DATASETS
- Secondly there is a an attempt by the trained model at a smooth transition between the latent variables at boundary and in the interior of the domain but the transition has only the lenght of the interior encoder mask radius. On way to fix this could be adding x,y distances to the outer boundary / clossest boundary into the interior encoder input data.

IMPORTANT NOTE: I have from the beggining just used very small training patches but it might be worth exploring If I could just solve a larger mesh problem or even the entire mesh during the training. This might have a lof of benefits